<a href="https://colab.research.google.com/github/Garvit-2301/MiniatureAgent_DSPy/blob/main/DSPyModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install dspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does n

In [2]:
import dspy
import json
import numpy as np
from typing import List, Dict, Any
from dataclasses import dataclass

lm = dspy.LM('model_name',api_key="your_api_key",api_base="",
    custom_llm_provider="")
dspy.configure(lm=lm,temperature = 0.7)

In [3]:
@dataclass
class HealthReport:
  """ Data for Health Reports """
  report_id: int
  similarity_score: int
  similarity_label: str
  underrepresented_species: List[str]
  health_score: float
  interpretation: str

@dataclass
class QuesAns:
  """ question-answer pairs """
  topic: str
  question: str
  answer: str

In [4]:
class Report_Summarizer(dspy.Signature):
  """ Summarize the Health Report """

  report_data = dspy.InputField(desc = "Health Report containing similarity and health scores, and species data")
  summary = dspy.OutputField(desc = "Brief summary focusing on key findings and also the implicaitons")

class Recommendation_Generator(dspy.Signature):
  """ Generate recommendations based on the analysis """

  health_score = dspy.InputField(desc="Health Score")
  underrepresented_species = dspy.InputField(desc="List of underrepresented species")
  similarity_score = dspy.InputField(desc="Similarity Score")

  recommendations = dspy.OutputField(desc="Recommendations based on the analysis, be it dietary, or supplement, or lifestyle recommendations")

class Species_Analyzer(dspy.Signature):
  """ Analysis of species and implications """

  species_list = dspy.InputField(desc="List of underrepresented species")
  health_score = dspy.InputField(desc="Health Score")

  health_issues = dspy.OutputField(desc="Metabolic issues and health implications")
  species_analysis = dspy.OutputField(desc="Analysis of species and implications")
  target_microbes = dspy.OutputField(desc="List of specific microbes to boost with brief rationale for each")

class Risk_Analyzer(dspy.Signature):
  """ Provide Insights about risks based on microbiome profile """

  health_score = dspy.InputField(desc="Health Score")
  underrepresented_species = dspy.InputField(desc="List of underrepresented species")
  similarity_score = dspy.InputField(desc="Similarity Score")

  risk = dspy.OutputField(desc="Risk assessment and health implications, no recommendations")

class Question_Valid(dspy.Signature):
  """ Checks whether the quesitons is related to the health reports """

  question = dspy.InputField(desc="Question asked by the user")
  data = dspy.InputField(desc="Reports and data fields available")

  valid = dspy.OutputField(desc="return 1 if question can be answered using only the available knowledge, otherwise return 0")
  why = dspy.OutputField(desc="Explain why the question is valid or invalid")

class Health_Score(dspy.Signature):
  """ provides health score and insights """

  health_score = dspy.InputField(desc="Health Score")

  score = dspy.OutputField(desc="Health score mentioned in the report")

class Similarity_Score_Info(dspy.Signature):
  """ provides similarity score, label, and insights """

  similarity_score = dspy.InputField(desc="Similarity Score")
  similarity_label = dspy.InputField(desc="Similarity Label")

  similarity_info = dspy.OutputField(desc="Similarity score and label mentioned in the report, along with a brief interpretation if available in the report.")

In [5]:
class HealthAgent(dspy.Module):
    """ Agent for Health Report Analysis and Recommendations - Context-Aware """

    def __init__(self):
        super().__init__()
        self.report_summarizer = dspy.ChainOfThought(Report_Summarizer)
        self.report_species_analyzer = dspy.ChainOfThought(Species_Analyzer)
        self.risk_analyzer = dspy.ChainOfThought(Risk_Analyzer)
        self.recommendations = dspy.ChainOfThought(Recommendation_Generator)
        self.is_valid = dspy.ChainOfThought(Question_Valid)
        self.health_score_module = dspy.ChainOfThought(Health_Score)
        self.similarity_module = dspy.ChainOfThought(Similarity_Score_Info)

        self.training_data = self.load_training_data()

        self.ques_patterns = {
            'summary': ['summary', 'summarize', 'brief', 'overview', 'takeaway', 'insights', 'what does this report show', 'top three takeaways'],
            'species': ['microbe', 'species', 'bacteria', 'underrepresented', 'missing bacteria', 'which bacteria', 'specific microbes', 'boost', 'target microbes'],
            'risk': ['risk', 'implications', 'diseases', 'condition', 'health implications', 'disrupted', 'damaged', 'what does this mean for my health', 'risk profile', 'disease states', 'pathways'],
            'recommendations': ['recommendations', 'diet', 'supplement', 'lifestyle', 'suggestions', 'advice', 'what should i do', 'how to improve', 'dietary changes', 'probiotic', 'consider'],
            'similarity_score': ['similarity score', 'similarity label', 'similarity value', 'similarity', 'my similarity score'],
            'health_score': ['health score', 'score', 'health label', 'health value'],
            'insights': ['insight', 'reveal', 'key insight', 'what insight'],
            'frequency': ['retest', 'frequently', 'how often', 'frequency']
        }

        self.available_data = [
            "similarity_score", "similarity_label", "health_score",
            "underrepresented_species", "interpretation", "report_id"
        ]

        self.reject_patterns = [
            'general health advice', 'other conditions', 'medications not mentioned', 'treatment plans',
            'supplements not related to microbiome', 'other symptoms', 'medical diagnosis',
            'personal questions', 'personal history', 'family history', 'genetic testing',
            'blood work', 'other lab tests', 'doctor visits', 'prescriptions',
            'weight loss', 'exercise routine', 'mental health', 'sleep patterns',
            'age related', 'pregnancy', 'children health', 'veterinary'
        ]

        self.rejection_message = "I can only answer questions about your health report data and provide insights similar to the examples in my training. Please ask about your similarity score, health score, underrepresented species, risks, or recommendations based on your specific report."

    def get_relevant_examples(self, question_type: str, report_id: int = None) -> str:
        """Get relevant examples from questioner.json for context"""

        examples = []
        for item in self.training_data['quesans']['questioner']:
            for qa in item['questions_answers']:
                topic = qa['topic'].lower()
                if (question_type == 'summary' and topic in ['summarization', 'insights']) or \
                   (question_type == 'species' and topic == 'species focus') or \
                   (question_type == 'risk' and topic == 'diagnosis') or \
                   (question_type == 'recommendations' and topic == 'recommendations') or \
                   (question_type == 'insights' and topic == 'insights') or \
                   (question_type == 'frequency' and topic == 'miscellaneous'):
                    examples.append(f"Q: {qa['question']}\nA: {qa['answer']}")

        return "\n\n".join(examples[:3])

    def validate(self, question: str, report: HealthReport) -> tuple[bool, str]:
        """Enhanced validation with context awareness"""
        question_lower = question.lower()

        for pattern in self.reject_patterns:
            if pattern in question_lower:
                return False, self.rejection_message

        report_keywords = [
            'similarity', 'health score', 'species', 'bacteria', 'microbe', 'report', 'analysis',
            'takeaway', 'insight', 'risk', 'recommendation', 'supplement', 'diet', 'probiotic',
            'boost', 'improve', 'retest', 'frequency', 'pathways', 'profile'
        ]
        has_report_keyword = any(keyword in question_lower for keyword in report_keywords)

        if not has_report_keyword:
            return False, self.rejection_message

        examples_context = self.get_relevant_examples('summary')
        result = self.is_valid(
            question=question,
            data=report,
            examples=examples_context,
            instruction="Only validate questions similar to the provided examples or directly about the health report data. Reject questions about general health, medications, or topics not covered in the examples."
        )

        if result.valid == '0':
            return False, self.rejection_message
        else:
            return True, "Valid question about health report"

    def classify(self, question: str) -> str:
        """Classify question type with enhanced patterns"""
        ques_lower = question.lower()

        # Check for specific patterns from questioner.json
        if 'takeaway' in ques_lower or 'top three' in ques_lower:
            return 'summary'
        elif 'insight' in ques_lower or 'reveal' in ques_lower:
            return 'insights'
        elif 'risk profile' in ques_lower or 'disease states' in ques_lower or 'pathways' in ques_lower:
            return 'risk'
        elif 'specific microbes' in ques_lower or 'boost' in ques_lower:
            return 'species'
        elif 'dietary changes' in ques_lower or 'probiotic' in ques_lower:
            return 'recommendations'
        elif 'retest' in ques_lower or 'frequently' in ques_lower:
            return 'frequency'

        for question_type, keywords in self.ques_patterns.items():
            if any(keyword in ques_lower for keyword in keywords):
                return question_type

        return 'summary'

    def load_training_data(self) -> Dict[str, Any]:
        reports_data = json.load(open("/content/drive/MyDrive/Resources and Projects/reports.json"))
        quesans_data = json.load(open("/content/drive/MyDrive/Resources and Projects/questioner.json"))
        return {"reports": reports_data, "quesans": quesans_data}

    def forward(self, report: HealthReport, question: str) -> str:
        """Main forward method with context-aware responses"""

        valid, why = self.validate(question, report)
        if not valid:
            return self.rejection_message

        question_type = self.classify(question)
        print(f"Classified question type: {question_type}")

        examples_context = self.get_relevant_examples(question_type, report.report_id)

        if question_type == 'summary':
            result = self.report_summarizer(
                report_data=report,
                examples=examples_context,
                instruction="Provide a summary based on the health report data, following the style and format of the provided examples. Focus only on the report data."
            )
            return f"Based on your health report: {result.summary}"

        elif question_type == 'species':
            result = self.report_species_analyzer(
                species_list=report.underrepresented_species,
                health_score=report.health_score,
                examples=examples_context,
                instruction="Analyze the underrepresented species based on the report data, following the style of the provided examples."
            )
            return f"According to your report: {result.species_analysis} {result.target_microbes}"

        elif question_type == 'risk':
            result = self.risk_analyzer(
                health_score=report.health_score,
                underrepresented_species=report.underrepresented_species,
                similarity_score=report.similarity_score,
                examples=examples_context,
                instruction="Analyze risks based on the specific report data, following the style of the provided examples."
            )

            species_result = self.report_species_analyzer(
                species_list=report.underrepresented_species,
                health_score=report.health_score,
                instruction="Provide health implications only for the species in this report."
            )

            return f"Based on your report data: {result.risk} {species_result.health_issues}"

        elif question_type == 'recommendations':
            result = self.recommendations(
                health_score=report.health_score,
                underrepresented_species=report.underrepresented_species,
                similarity_score=report.similarity_score,
                examples=examples_context,
                instruction="Provide recommendations based on the specific report data, following the style of the provided examples."
            )
            return f"Based on your report results: {result.recommendations}"

        elif question_type == 'similarity_score':
            result = self.similarity_module(
                similarity_score=report.similarity_score,
                similarity_label=report.similarity_label,
                instruction="Provide similarity score and label from the report."
            )
            return f"Based on your report: {result.similarity_info}"

        elif question_type == 'health_score':
            result = self.health_score_module(
                health_score=report.health_score,
                instruction="Explain the health score based only on the report data."
            )
            return f"Based on your health report: Your Health Score is {result.score}"

        elif question_type == 'insights':
            result = self.report_summarizer(
                report_data=report,
                examples=examples_context,
                instruction="Provide key insights based on the health report data, following the style of the provided examples."
            )
            return f"Key insight from your report: {result.summary}"

        elif question_type == 'frequency':
            return "Based on typical microbiome monitoring practices, retesting every 3-4 months is recommended to track the effectiveness of interventions."

        else:
            result = self.report_summarizer(
                report_data=report,
                examples=examples_context,
                instruction="Provide a summary based on the health report data."
            )
            return f"Based on your health report: {result.summary}"

In [6]:
class HealthAgentTraining:
  """ Training module for the health agent """
  def __init__(self, agent: HealthAgent):
    self.agent = agent
    self.training_data = self.load_training_data()

  def load_training_data(self) -> Dict[str, Any]:
    reports_data = json.load(open("/content/drive/MyDrive/Resources and Projects/reports.json"))
    quesans_data = json.load(open("/content/drive/MyDrive/Resources and Projects/questioner.json"))
    return {"reports": reports_data, "quesans": quesans_data}


  def train(self, training_data: Dict[str, Any]):
    optimizer = dspy.BootstrapFewShot(
        metric=self.answer_quality_metric,
        max_bootstrapped_demos=4,
        max_labeled_demos=8

    )

    agent_compile = optimizer.compile(
        self.agent,
        trainset = training_data
    )

    return agent_compile

  def metric(self, example, pred, trace=None):

    expected_keywords = set(example.expected_answer.lower().split())
    predicted_keywords = set(pred.answer.lower().split())

    overlap = len(expected_keywords.intersection(predicted_keywords))
    union = len(expected_keywords.union(predicted_keywords))

    return overlap / union if union > 0 else 0

In [9]:
agent = HealthAgent()
training_module = HealthAgentTraining(agent)

sample_report = HealthReport(
        report_id=2,
        similarity_score=0.394,
        similarity_label="Moderately different",
        underrepresented_species=[
            "Collinsella aerofaciens",
          "Lactobacillus rhamnosus",
          "Alistipes onderdonkii",
          "Ruminococcus bromii",
          "Eubacterium rectale",
          "Bifidobacterium breve"
        ],
        health_score=0.131,
        interpretation="Slightly skewed toward healthy, though the presence of underrepresented beneficial microbes indicates a vulnerable balance. Continued monitoring and dietary fiber intake is recommended to enhance diversity."
)

question = str(input("Hi! How can I help you? "))
response = agent.forward(sample_report,question)
display(response)

Hi! How can I help you? what is my health score ?
Classified question type: health_score


'Based on your health report: Your Health Score is 0.131'